In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/'Colab Notebooks'/'yai_9기 겨울방학'/ToyProject

/content/drive/MyDrive/Colab Notebooks/yai_9기 겨울방학/ToyProject


In [4]:
csv_path = "./final.csv"
image_path = "./data/images"
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [5]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [6]:
class AllDataset(Dataset):
    def __init__(self, csv_path, image_path, image_transform=transforms.Compose([transforms.ToTensor()])):
        super(AllDataset).__init__()
        csv = pd.read_csv(csv_path)
        csv_np = csv.to_numpy()
        self.images = []
        for Id in csv['Id']:
            Id = str(Id)
            image = Image.open(image_path+'/'+Id+'.jpg')
            self.images.append(image_transform(image))
            image.close()
        
        for i in [1, 3, 4, 5, 6, 7, 8]:
            wordset = {word: idx for idx, word in enumerate(np.unique(csv_np[:,i]))}
            for row in range(len(csv_np)):
                csv_np[row][i] = wordset[csv_np[row][i]]
        self.ints = torch.from_numpy(np.array(csv_np[:,[1,3,4,5,6,7,8]], dtype="int"))
        self.floats = torch.from_numpy(np.array(list(map(lambda x: (x - np.array([0.61756694, 5.49324267, 4.76044624, 3.05992563])) / np.array([0.39076653, 1.28855836, 0.77676551, 0.51616518]), csv_np[:,[2,9,10,11]])), dtype="float")).float()
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
    
    
    def __getitem__(self,idx):
        return self.images[idx], self.ints[idx],self.floats[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.ints)

In [7]:
data_length = len(pd.read_csv(csv_path))
train_length = int(data_length * 0.6)
test_length = int(data_length * 0.2)
val_length = data_length - train_length - test_length

image_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5),
                                                         (0.5, 0.5, 0.5))
                                     ])

train_dataset = AllDataset(csv_path, image_path, image_transform)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=False)
        self.resnet18.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet18.fc = nn.Sequential(nn.Linear(512, 2048),
                                         nn.ReLU(),
                                        nn.Linear(2048, 2048),
                                        nn.ReLU())
        self.resenet18 = self.resnet18.to(device)
        
        self.emb1 = torch.nn.Embedding(8, 20)
        self.emb2 = torch.nn.Embedding(11, 20)
        self.emb3 = torch.nn.Embedding(20, 20)
        self.emb4 = torch.nn.Embedding(4, 20)
        self.emb5 = torch.nn.Embedding(4, 20)
        self.emb6 = torch.nn.Embedding(4, 20)
        self.emb7 = torch.nn.Embedding(7, 20)
        self.act = nn.ReLU()
        self.fc = nn.Linear(4, 80)
        self.csvfc1 = nn.Linear(220, 4096)
        self.csvfc2 = nn.Linear(4096, 4096)
        self.csvfc3 = nn.Linear(4096, 2048)
        
        self.fc1 = nn.Linear(4096, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 1)
        self.dropout = nn.Dropout()
    
    def forward(self, image, x, y):
        image = self.resnet18(image)
        
        x1 = self.emb1(x[:,0])
        x2 = self.emb2(x[:,1])
        x3 = self.emb3(x[:,2])
        x4 = self.emb4(x[:,3])
        x5 = self.emb5(x[:,4])
        x6 = self.emb6(x[:,5])
        x7 = self.emb7(x[:,6])
        y = self.fc(y)
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7, y), dim=1)
        
        x = self.dropout(self.act(self.csvfc1(x)))
        x = self.dropout(self.act(self.csvfc2(x)))
        x = self.dropout(self.act(self.csvfc3(x)))
        x = torch.cat((x, image), dim=1)
        
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        return self.fc3(x)

In [18]:
model = Net().to(device)

epochs = 30
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for Image, X1, X2, Y in train_loader:
        Image = Image.to(device)
        X1 = X1.to(device)
        X2 = X2.to(device)
        Y = Y.to(device)

        model.zero_grad()  # why we use zero_grad?
        prediction = model(Image, X1, X2)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for Image_val, X1_val, X2_val, Y_val in val_loader:
            Image_val = Image_val.to(device)
            X1_val = X1_val.to(device)
            X2_val = X2_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(Image_val, X1_val, X2_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 4176.3203, val_loss: 3000.7666
[Epoch:  2] Average loss: 2618.1111, val_loss: 1926.8380
[Epoch:  3] Average loss: 2235.9814, val_loss: 3747.4629
[Epoch:  4] Average loss: 2363.0737, val_loss: 2044.1440
[Epoch:  5] Average loss: 1840.7393, val_loss: 2205.0020
[Epoch:  6] Average loss: 1554.2322, val_loss: 1646.1033
[Epoch:  7] Average loss: 1420.6560, val_loss: 1902.5278
[Epoch:  8] Average loss: 1402.7977, val_loss: 1333.4504
[Epoch:  9] Average loss: 1658.4451, val_loss: 1396.8569
[Epoch: 10] Average loss: 1538.9701, val_loss: 2212.9727
[Epoch: 11] Average loss: 1805.0626, val_loss: 1767.2482
[Epoch: 12] Average loss: 1360.6362, val_loss: 1438.3768
[Epoch: 13] Average loss: 1171.2269, val_loss: 1490.0527
[Epoch: 14] Average loss: 1162.5975, val_loss: 1681.0963
[Epoch: 15] Average loss: 1005.2097, val_loss: 1895.9912
[Epoch: 16] Average loss: 1389.1746, val_loss: 1607.2794
[Epoch: 17] Average loss: 1257.2994, val_loss: 1647.3928
[Epoch: 18] Average loss: 1087.

In [23]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
test_loader = DataLoader(test_dataset, batch_size=1, shuffle = True)
pred = []
label = []
with torch.no_grad():
    for Image_test, X1_test, X2_test, Y_test in test_loader:
        if Y_test < 10000:
            continue
        Image_test =Image_test.to(device)
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(Image_test, X1_test, X2_test)
        print(f"결과값, 예측값 : {Y_test}, {prediction}")
        pred.append(prediction)
        label.append(Y_test)
    pred = torch.tensor(pred)
    label = torch.tensor(label)
    mean = torch.mean(label)
    ss_tot += torch.sum((label - mean) ** 2)
    ss_res += torch.sum((label - pred) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    print(f"Accuracy: {accuracy*100:.2f}%")

결과값, 예측값 : tensor([[15504.2900]], device='cuda:0'), tensor([[20114.2852]], device='cuda:0')
결과값, 예측값 : tensor([[16247.8701]], device='cuda:0'), tensor([[20697.0898]], device='cuda:0')
결과값, 예측값 : tensor([[13595.2100]], device='cuda:0'), tensor([[14647.5156]], device='cuda:0')
결과값, 예측값 : tensor([[64733.5195]], device='cuda:0'), tensor([[68685.9141]], device='cuda:0')
결과값, 예측값 : tensor([[13336.8604]], device='cuda:0'), tensor([[18803.2188]], device='cuda:0')
결과값, 예측값 : tensor([[18073.7402]], device='cuda:0'), tensor([[22250.6895]], device='cuda:0')
결과값, 예측값 : tensor([[12820.0898]], device='cuda:0'), tensor([[14212.4375]], device='cuda:0')
결과값, 예측값 : tensor([[18681.6406]], device='cuda:0'), tensor([[19013.1699]], device='cuda:0')
결과값, 예측값 : tensor([[41056.6484]], device='cuda:0'), tensor([[31168.8574]], device='cuda:0')
결과값, 예측값 : tensor([[13019.6904]], device='cuda:0'), tensor([[19895.0762]], device='cuda:0')
결과값, 예측값 : tensor([[68738.5234]], device='cuda:0'), tensor([[40838.1055]], devic

전체 price : 84.68 %
10000이상 price : 73.31%
하지만 eda 보시면 10000~ 20000도 정상적인 범주로 들어가서
만약 price가 30000 이상인 데이터만 또 다루면 metric이 떨어질 것으로 예상
epoch는 동일하게 두고 비교한 것입니다.